# Projekt MAGN

### Autorzy:

- Szymon Jurecki
- Dominik Breksa

In [1]:
%pip install .
%pip install pandas
%pip install numpy
%pip install kaggle

Processing c:\users\domin\pycharmprojects\ggsn---magn
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for magn: filename=magn-1.0.0.0-py3-none-any.whl size=16641 sha256=7499b71291deb44830891e32c82713c400470f486fcf4e79d89238e2c1603885
  Stored in directory: c:\users\domin\appdata\local\pip\cache\wheels\02\69\7a\7a321fc99cdf850f17c5dbe96e53ef1e4b07adda9d5bd3d1f7
Successfully built magn
  Attempting uninstall: magn
    Found existing installation: magn 1.0.0.0
    Not uninstalling magn at c:\users\domin\pycharmprojects\ggsn---magn\src, outside environment C:\Users\domin\PycharmProjects\GGSN--

DEPRECATION: Loading egg at c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages\magn-1.0.0.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages\magn-1.0.0.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: numpy in c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages (1.26.4)



DEPRECATION: Loading egg at c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages\magn-1.0.0.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages\magn-1.0.0.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Pobranie oraz Przygotowanie Danych

In [2]:
from pathlib import Path
from typing import Final

resources_dir: Final[Path] = Path('resources/data/')

zip_data_file_movie: Final[Path] = resources_dir.joinpath('movie-data-analytics-dataset.zip')
database_path_movie: Final[Path]  = resources_dir.joinpath('movie.sqlite')

zip_data_file_pitchfork: Final[Path] = resources_dir.joinpath('pitchfork-data.zip')
database_path_pitchfork: Final[Path]  = resources_dir.joinpath('database.sqlite')

Pobranie danych z platformy Kaggle

[Link do danych z platformy Kaggle](https://www.kaggle.com/datasets/shahjhanalam/movie-data-analytics-dataset)

[Link do danych z platformy Kaggle](https://www.kaggle.com/datasets/nolanbconaway/pitchfork-data)

In [3]:
from os.path import exists

if not exists(zip_data_file_movie):
    !kaggle datasets download -d shahjhanalam/movie-data-analytics-dataset -p resources/data
    
if not exists(zip_data_file_pitchfork):
    !kaggle datasets download -d nolanbconaway/pitchfork-data -p resources/data

Rozpakowanie danych do odpowiedniego katalogu

In [4]:
from zipfile import ZipFile

if not exists(database_path_movie):
    with ZipFile(zip_data_file_movie, 'r') as zip_ref:
        zip_ref.extractall(resources_dir)

if not exists(database_path_pitchfork):
    with ZipFile(zip_data_file_pitchfork, 'r') as zip_ref:
        zip_ref.extractall(resources_dir)

Funkcje pomocnicze do obsługi bazy danych SQLite3

In [5]:
from typing import List, Tuple
import sqlite3

def get_table_columns_types(db_cursor: sqlite3.Cursor, table_name: str) -> List[Tuple[str, str]]:
    """Returns a list of tuples containing the column name and data type of given table in a database."""

    info = db_cursor.execute(f"""
        PRAGMA
            table_info({table_name});
    """).fetchall()

    return list((table_info[1], table_info[2]) for table_info in info)

def get_table_names(db_cursor: sqlite3.Cursor) -> List[str]:
    """"""
    db_cursor.execute("""
        SELECT
            name
        FROM
            sqlite_master
        WHERE
            type='table'
    """)

    return list(table[0] for table in db_cursor.fetchall())

def create_table_with_primary_key(db_cursor: sqlite3.Cursor, table_name: str, temporary_table_name: str, column_name: str) -> None:
    """Creates a table with a primary key in a database."""

    table_columns = get_table_columns_types(db_cursor, table_name)

    db_cursor.execute(f"""
        CREATE TABLE {temporary_table_name} (
            {
                ', '.join(
                    f'{column} {data_type}'
                    if column != column_name
                    else f'{column} {data_type} PRIMARY KEY'
                    for column, data_type in table_columns
                )
            }
        );
    """)

def move_data_and_switch_tables(db_cursor: sqlite3.Cursor, table_name: str, temporary_table_name: str) -> None:
    """Moves data from a temporary table to the original table and switches the tables in a database."""

    db_cursor.executescript(f"""
        -- Move the data
        INSERT INTO
            {temporary_table_name}
        SELECT
            *
        FROM
            {table_name};
        
        -- Drop the old table
        DROP TABLE
            {table_name};
        
        -- Rename the new table
        ALTER TABLE
            {temporary_table_name}
        RENAME TO
            {table_name};
    """)

def create_table_with_foreign_key(db_cursor: sqlite3.Cursor, table_name: str, column_name: str, foreign_table_name: str, foreign_column_name: str, temporary_foreign_table_name: str) -> None:
    """Creates a table with a foreign key in a database."""

    foreign_table_columns = get_table_columns_types(db_cursor, foreign_table_name)

    db_cursor.execute(f"""
        CREATE TABLE {temporary_foreign_table_name} (
            {
                ', '.join(
                    f'{column} {data_type}'
                    for column, data_type in foreign_table_columns
                )
            },
            CONSTRAINT fk_{foreign_table_name}_{foreign_column_name}
                FOREIGN KEY
                    ({foreign_column_name})
                REFERENCES
                    {table_name} ({column_name})
        );
    """)

Limitowanie danych w tabelach do 1000 rekordów

In [6]:
def delete_duplicates(db_cursor: sqlite3.Cursor, table_name: str, column_name: str) -> None:
    """Deletes duplicates from a given table column in a database."""


    for given_table_name in get_table_names(db_cursor):
        for given_column_name, _ in get_table_columns_types(db_cursor, given_table_name):
            if column_name == given_column_name:
                db_cursor.execute(f"""
                    DELETE FROM
                        {given_table_name}
                    WHERE
                        {given_column_name} IN (
                            SELECT
                                {column_name}
                            FROM
                                {table_name}
                            GROUP BY
                                {column_name}
                            HAVING
                                COUNT(*) > 1
                        );
                """)
            else:
                db_cursor.execute(f"""
                    DELETE FROM
                        {given_table_name}
                    WHERE
                        {given_column_name} IS NULL;
                """)

    db_cursor.execute(f"""
        DELETE FROM
            {table_name}
        WHERE
            {column_name} NOT IN (
                {
    'INTERSECT'.join(f"""
                            SELECT
                                {column_name}
                            FROM
                                {given_table_name}
                        """
                     for given_table_name in get_table_names(db_cursor)
                     )
    }
            );
    """)

    for given_table_name in get_table_names(db_cursor):
        db_cursor.execute(f"""
            DELETE FROM
                {given_table_name}
            WHERE
                {column_name} NOT IN (
                    SELECT
                        {column_name}
                    FROM
                        {table_name}
                );
        """)

In [7]:
def limit_rows(db_cursor: sqlite3.Cursor, column_name: str, max_rows: int) -> None:
    """Limits the number of rows in a table in a database."""
    
    all_tables = get_table_names(db_cursor)
    
    # From every table delete rows that are a duplicate of the column_name
    for table_name in all_tables:
        db_cursor.execute(f"""
            DELETE FROM
                {table_name}
            WHERE
                {column_name} IN (
                    SELECT
                        {column_name}
                    FROM
                        {table_name}
                    GROUP BY
                        {column_name}
                    HAVING
                        COUNT(*) > 1
                );
        """)
    
    # Find the intersection of a column_name in every table
    # Delete rows that are not in the intersection
    intersection: str = 'INTERSECT'.join(f"""
        SELECT
            {column_name}
        FROM
            {intersection_table_name}
    """ for intersection_table_name in all_tables)
    
    for table_name in all_tables:
        db_cursor.execute(f"""
            DELETE FROM
                {table_name}
            WHERE
                {column_name} NOT IN (
                    {intersection}
                );
        """)
    
    # Limit the number of rows in a table to max_rows
    for table_name in all_tables:
        db_cursor.execute(f"""
            DELETE FROM
                {table_name}
            WHERE
                {column_name} NOT IN (
                    SELECT
                        {column_name}
                    FROM
                        {table_name}
                    ORDER BY
                        {column_name}
                    LIMIT {max_rows}
                );
        """)

In [8]:
from typing import Final

MAX_ROWS: Final[int] = 1000

with sqlite3.connect(database_path_movie) as conn:
    limit_rows(conn.cursor(), 'Movie_id', MAX_ROWS)

In [9]:
with sqlite3.connect(database_path_pitchfork) as conn:
    limit_rows(conn.cursor(), 'reviewid', MAX_ROWS)

Dodanie kluczy głównych i obcych do bazy danych, ponieważ nie zostały one dodane podczas tworzenia na platformie Kaggle, a jest to ważne podczas tworzenia MAGN.

In [10]:
def add_foreign_key(db_cursor: sqlite3.Cursor, table_name: str, column_name: str, foreign_table_name: str, foreign_column_name: str) -> None:
    """Adds a foreign key to a given table column in a database."""

    temporary_foreign_table_name = f'{foreign_table_name}_'

    db_cursor.execute(f"""
        PRAGMA
            foreign_keys = OFF;
    """)

    create_table_with_foreign_key(db_cursor, table_name, column_name, foreign_table_name, foreign_column_name, temporary_foreign_table_name)
    move_data_and_switch_tables(db_cursor, foreign_table_name, temporary_foreign_table_name)

    db_cursor.execute(f"""
        PRAGMA
            foreign_keys = ON;
    """)

def add_primary_key(db_cursor: sqlite3.Cursor, table_name: str, column_name: str) -> None:
    """Adds a primary key to a given table column in a database."""

    temporary_table_name = f'{table_name}_'

    create_table_with_primary_key(db_cursor, table_name, temporary_table_name, column_name)
    move_data_and_switch_tables(db_cursor, table_name, temporary_table_name)

In [11]:
with sqlite3.connect(database_path_movie) as conn:
    cursor = conn.cursor()

    cursor.execute(f"""BEGIN;""")
    try:
        add_primary_key(cursor, 'IMDB', 'Movie_id')

        add_foreign_key(cursor, 'IMDB', 'Movie_id', 'earning', 'Movie_id')
        add_foreign_key(cursor, 'IMDB', 'Movie_id', 'genre', 'Movie_id')
    except Exception as error:
        conn.rollback()
        raise error
    else:
        conn.commit()

![ERD Diagram for the database schema](docs/images/database_erd_movie.png_)

In [12]:
with sqlite3.connect(database_path_pitchfork) as conn:
    cursor = conn.cursor()

    cursor.execute(f"""BEGIN;""")
    try:
        add_primary_key(cursor, 'reviews', 'reviewid')

        add_foreign_key(cursor, 'reviews', 'reviewid', 'artists', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'content', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'genres', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'labels', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'years', 'reviewid')
    except Exception as error:
        conn.rollback()
        raise error
    else:
        conn.commit()

![ERD Diagram for the database schema](docs/images/database_erd_pitchfork.png_)

In [13]:
from magn.database.sqlite3 import get_table_names

all_tables_movie = get_table_names(database_path_movie)

all_tables_movie

['IMDB', 'earning', 'genre']

In [14]:
all_tables_pitchfork = get_table_names(database_path_pitchfork)

all_tables_pitchfork

['reviews', 'artists', 'content', 'genres', 'labels', 'years']

In [15]:
from magn.database.sqlite3 import SQLite3KeysReader

keys_reader = SQLite3KeysReader(database_path_movie, all_tables_movie)
keys_movie = keys_reader.read()

keys_movie

{'IMDB': Keys(primary_keys=['Movie_id'], foreign_keys={}),
 'earning': Keys(primary_keys=[], foreign_keys={'IMDB': ('Movie_id', 'Movie_id')}),
 'genre': Keys(primary_keys=[], foreign_keys={'IMDB': ('Movie_id', 'Movie_id')})}

In [16]:
keys_reader = SQLite3KeysReader(database_path_pitchfork, all_tables_pitchfork)
keys_pitchfork = keys_reader.read()

keys_pitchfork

{'reviews': Keys(primary_keys=['reviewid'], foreign_keys={}),
 'artists': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'content': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'genres': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'labels': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'years': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')})}

In [17]:
from magn.database.sqlite3 import SQLite3DataReader

data_reader = SQLite3DataReader(database_path_movie, all_tables_movie, keys_movie)
data_reader.read()

{'IMDB': Empty DataFrame
 Columns: [Title, Rating, TotalVotes, MetaCritic, Budget, Runtime, CVotes10, CVotes09, CVotes08, CVotes07, CVotes06, CVotes05, CVotes04, CVotes03, CVotes02, CVotes01, CVotesMale, CVotesFemale, CVotesU18, CVotesU18M, CVotesU18F, CVotes1829, CVotes1829M, CVotes1829F, CVotes3044, CVotes3044M, CVotes3044F, CVotes45A, CVotes45AM, CVotes45AF, CVotes1000, CVotesUS, CVotesnUS, VotesM, VotesF, VotesU18, VotesU18M, VotesU18F, Votes1829, Votes1829M, Votes1829F, Votes3044, Votes3044M, Votes3044F, Votes45A, Votes45AM, Votes45AF, VotesIMDB, Votes1000, VotesUS, VotesnUS]
 Index: []
 
 [0 rows x 51 columns],
 'earning': Empty DataFrame
 Columns: [Domestic, Worldwide]
 Index: [],
 'genre': Empty DataFrame
 Columns: [genre]
 Index: []}

In [18]:
data_reader = SQLite3DataReader(database_path_pitchfork, all_tables_pitchfork, keys_pitchfork)
data_reader.read()

{'reviews':                                                       title  \
 reviewid                                                      
 1                                             young forever   
 6                                      pure tone audiometry   
 7         or you could just go through your whole life a...   
 8                                            bleeding light   
 10                                                     abcs   
 ...                                                     ...   
 2398             goodbye enemy airship the landlord is dead   
 2399                                            & yet & yet   
 2400                   winter hymn country hymn secret hymn   
 2403                              will you still love me ep   
 2405                                        heart and crime   
 
                      artist  \
 reviewid                      
 1                 aberfeldy   
 6                  aarktica   
 7                  aarktic

In [19]:
from magn.database.database import Database

database_movie = Database.from_sqlite3(database_path_movie)

database_movie

Database(all_data={'IMDB': Table(data=Empty DataFrame
Columns: [Title, Rating, TotalVotes, MetaCritic, Budget, Runtime, CVotes10, CVotes09, CVotes08, CVotes07, CVotes06, CVotes05, CVotes04, CVotes03, CVotes02, CVotes01, CVotesMale, CVotesFemale, CVotesU18, CVotesU18M, CVotesU18F, CVotes1829, CVotes1829M, CVotes1829F, CVotes3044, CVotes3044M, CVotes3044F, CVotes45A, CVotes45AM, CVotes45AF, CVotes1000, CVotesUS, CVotesnUS, VotesM, VotesF, VotesU18, VotesU18M, VotesU18F, Votes1829, Votes1829M, Votes1829F, Votes3044, Votes3044M, Votes3044F, Votes45A, Votes45AM, Votes45AF, VotesIMDB, Votes1000, VotesUS, VotesnUS]
Index: []

[0 rows x 51 columns], keys=Keys(primary_keys=['Movie_id'], foreign_keys={})), 'earning': Table(data=Empty DataFrame
Columns: [Domestic, Worldwide]
Index: [], keys=Keys(primary_keys=[], foreign_keys={'IMDB': ('Movie_id', 'Movie_id')})), 'genre': Table(data=Empty DataFrame
Columns: [genre]
Index: [], keys=Keys(primary_keys=[], foreign_keys={'IMDB': ('Movie_id', 'Movie_id'

In [20]:
database_pitchfork = Database.from_sqlite3(database_path_pitchfork)

database_pitchfork

Database(all_data={'reviews': Table(data=                                                      title  \
reviewid                                                      
1                                             young forever   
6                                      pure tone audiometry   
7         or you could just go through your whole life a...   
8                                            bleeding light   
10                                                     abcs   
...                                                     ...   
2398             goodbye enemy airship the landlord is dead   
2399                                            & yet & yet   
2400                   winter hymn country hymn secret hymn   
2403                              will you still love me ep   
2405                                        heart and crime   

                     artist  \
reviewid                      
1                 aberfeldy   
6                  aarktica   
7              

In [21]:
database_pitchfork.create_mock_target("reviews")

,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year,target
reviewid,,,,,,,,,,,,,
1,young forever,aberfeldy,http://pitchfork.com/reviews/albums/1-young-fo...,3.4,0,marc hogan,senior staff writer,2004-08-30,0,30,8,2004,author_type
6,pure tone audiometry,aarktica,http://pitchfork.com/reviews/albums/6-pure-ton...,7.4,0,eric carr,contributor,2003-04-08,1,8,4,2003,author_type
7,or you could just go through your whole life a...,aarktica,http://pitchfork.com/reviews/albums/7-or-you-c...,8.1,0,christopher dare,None,2002-05-16,3,16,5,2002,title
8,bleeding light,aarktica,http://pitchfork.com/reviews/albums/8-bleeding...,7.2,0,sam ubl,None,2005-02-06,6,6,2,2005,best_new_music
10,abcs,abcs,http://pitchfork.com/reviews/albums/10-abcs/,8.0,0,michael bernstein,None,2002-05-07,1,7,5,2002,pub_weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2398,goodbye enemy airship the landlord is dead,do make say think,http://pitchfork.com/reviews/albums/2398-goodb...,7.9,0,s. murray,None,2000-03-28,1,28,3,2000,title
2399,& yet & yet,do make say think,http://pitchfork.com/reviews/albums/2399-yet-yet/,8.1,0,joe tangari,contributor,2002-04-04,3,4,4,2002,best_new_music
2400,winter hymn country hymn secret hymn,do make say think,http://pitchfork.com/reviews/albums/2400-winte...,8.1,0,joe tangari,contributor,2003-10-15,2,15,10,2003,pub_month
